<a href="https://colab.research.google.com/github/cis9650group7-boop/Group-7_Project1_HealthRateAnalysis/blob/dev/term_project_group7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1: Analysis of Health Rate

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Author: CIS 9650 Group 7 (Anish Bijusoman, Ivana Sundararao, Qingrong Tan, Reem Hussein)
### Date : November 28th, 2025

## Executive Summary

## Table of Contents

1. Introduction
2. Problem Statement / Research Question
3. Data Description
4. Setup and Environment
5. Data Loading
6. Data Preparation
7. Model Planning
8. Model Building / Analysis
9. Discussion & Interpretation
10. Conclusion
11. References
12. Appendix

## Introduction

## Problem Statement / Research Question

Predicting the overall quality of a clinic or hospital based on the location

## Data Description

- 136,000+ unique healthcare locations, each with metadata such as geolocation, ratings, contact details, and images.

- Doctor-to-location relationships, where each doctor may be linked to multiple clinic/hospital locations (up to 22).

- City and province information, which is highly varied (1,511 unique cities) but shows hierarchical grouping.

- Doctor location hours, providing structured scheduling data with attributes like day of week, opening/closing times, and timezone.

- Despite the richness of the dataset, it suffers from high cardinality, nested structures, multi-level relationships, and significant sparsity in some fields (e.g., images, postal codes, ratings).

- These issues make it difficult to efficiently perform:
1. Data quality analysis
2. Location search and ranking
3. Provider attribution
4. Geographic clustering
5. Operational hours normalization
6. Predictive modeling based on ratings or availability

## Setup and Environment

In [17]:
!pip install google-cloud-storage

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import json
import os
from google.cloud import storage
from io import BytesIO

## Data Loading

### Functions

In [6]:
from google.cloud import storage
import pandas as pd
from io import BytesIO
import os
import json
import requests
import re # Import regex module for parsing HTML

def list_public_bucket(bucket_name, course, project):
    prefix = f"{course}/Project {project}/"

    client = storage.Client.create_anonymous_client()
    bucket = client.bucket(bucket_name)
    blobs = client.list_blobs(bucket, prefix=prefix)
    print(f"Listing files in public bucket '{bucket_name}' under '{prefix}':")
    filenames = []
    for blob in blobs:
        print(blob.name)
        filenames.append(blob.name)

    return filenames

def gdrive_file_to_dataframe(file_id: str, file_type: str):
    """
    Downloads a file from Google Drive using its ID and returns a pandas DataFrame.
    Handles Google Drive virus scan warnings by parsing the HTML form to get the
    actual download URL.
    """
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()
    response = session.get(URL, params={'id': file_id}, stream=True)
    response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)

    # Check if the response content is the virus scan warning HTML page
    if "Google Drive - Virus scan warning" in response.text:
        print("Google Drive virus scan warning detected. Attempting to bypass...")
        # Extract the download action URL and hidden input parameters from the HTML form
        match = re.search(r'<form id="download-form" action="([^"]+)" method="get">', response.text)
        if not match:
            raise ValueError("Could not find download form in virus warning page.")
        download_action_url = match.group(1)

        hidden_inputs = re.findall(r'<input type="hidden" name="([^"]+)" value="([^"]*)">', response.text)
        download_params = {name: value for name, value in hidden_inputs}

        # Make the actual download request with the extracted parameters
        final_response = session.get(download_action_url, params=download_params, stream=True)
        final_response.raise_for_status()
        data = BytesIO(final_response.content)
    else:
        # No virus warning, proceed with the initial response content
        data = BytesIO(response.content)

    df = None
    if file_type == "csv":
        df = pd.read_csv(data, engine='python', on_bad_lines='warn')
    elif file_type in ["xls", "xlsx"]:
        df = pd.read_excel(data)
    elif file_type == "parquet":
        df = pd.read_parquet(data)
    elif file_type == "json":
        text = data.read().decode("utf-8")
        json_obj = json.loads(text)
        if isinstance(json_obj, list):
            df = pd.DataFrame(json_obj)
        elif isinstance(json_obj, dict):
            df = pd.json_normalize(json_obj)
        else:
            raise ValueError("Unsupported JSON structure")
    else:
        raise ValueError(f"Unsupported file type: {file_type}")

    return df

# Fetching the data from the provided Google Drive link
try:
    file_id_to_fetch = '1yJjVih68D_J4JON2LXMLGIh_FoeibwaG'
    # Assuming the file type is CSV based on the filename provided
    gdrive_df = gdrive_file_to_dataframe(file_id_to_fetch, 'csv')
    print("Successfully fetched and loaded data from Google Drive into 'gdrive_df'. Head of the DataFrame:")
    print(gdrive_df.head())
except Exception as e:
    print(f"Error fetching or loading data from Google Drive: {e}")


Google Drive virus scan warning detected. Attempting to bypass...
Successfully fetched and loaded data from Google Drive into 'gdrive_df'. Head of the DataFrame:
                        _id                                           slug  \
0  647e89e399dca372f92a0d7c    3183031/Dr-Janan+S.-Sayyed-NEW+YORK-NY.html   
1  647e89e499dca372f92a0d91      3345163/Dr-SHIRIN-PETERS-NEW+YORK-NY.html   
2  647e89e499dca372f92a0db9  3081147/Dr-THOMAS+J.-MULHERN-New+York-NY.html   
3  647e89e499dca372f92a0dba      3081144/Dr-ZINA-RUTKIN-GREAT+NECK-NY.html   
4  647e89e499dca372f92a0dbd  3081128/Dr-JOHN+S.-CAVALLARO-Brooklyn-NY.html   

                            created                          modified      id  \
0  2014-06-13T11:39:19.399618-04:00  2023-04-16T10:17:11.091386-04:00  870323   
1  2014-06-13T10:53:51.310523-04:00  2022-11-15T15:19:02.430196-05:00  819737   
2  2014-06-13T12:33:17.025883-04:00  2022-03-08T14:16:12.357691-05:00  939574   
3  2014-06-13T12:33:17.131851-04:00  2022-03-

## Data Preparation

In [ ]:
df = gdrive_df

print("Number of rows", len(df))
print("Number of columns", len(df.columns))

In [ ]:
# datatypes available
print("The datatypes available are:")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(df.dtypes)

In [ ]:
# similar naming columns
pd.set_option('display.max_columns', None)
df.columns = df.columns.str.lower().str.replace('_', '')

list_cols= list(df.columns)
print("The columns are:")
print(list_cols)

These need not be changed:'id', 'slug', 'created', 'modified', 'id', 'faceturl', 'fullname', 'fullnamespecialty'

# Method 1: Feature engineering

In [ ]:
# # Select usable columns
# cols_to_keep = ["_id", "full_name", "specialty_name", "location.city.name","rating.average", "rating.helpfulness", "rating.punctuality","rating.staff", "rating.count"]

# df_clean = df[cols_to_keep].copy()

# # Rename columns to simpler names
# df_clean = df_clean.rename(columns={
#     "location.city.name": "city",
#     "rating.average": "rating_avg",
#     "rating.helpfulness": "rating_help",
#     "rating.punctuality": "rating_punctuality",
#     "rating.staff": "rating_staff",
#     "rating.count": "rating_count"
# })

# # Convert numeric columns to proper numeric types
# numeric_cols = [
#     "rating_avg", "rating_help", "rating_punctuality",
#     "rating_staff", "rating_count"
# ]

# for col in numeric_cols:
#     df_clean[col] = pd.to_numeric(df_clean[col], errors="coerce")

# # Drop rows with no rating information
# df_clean = df_clean.dropna(subset=["rating_avg"])

# # Before having this the rating avg and rating_help was 0's
# df_clean = df_clean[df_clean["rating_count"] > 0]
# df_clean = df_clean[df_clean["rating_avg"] > 0]

# # Reset index
# df_clean = df_clean.reset_index(drop=True)

# df_clean.head() # this is to see the clean data

# Method 2: Feature engineering

Most columns share prefixes like:
location.*
images.*
rating.*
doctorlocations[n].*
doctorlocationhours[n].*

We can automatically group these

In [ ]:
import pandas as pd

def group_columns_by_prefix(df, sep='.'):
    groups = {}
    for col in df.columns:
        prefix = col.split(sep)[0]
        groups.setdefault(prefix, []).append(col)
    return groups

groups = group_columns_by_prefix(df)
for g, cols in groups.items():
    print(g, len(cols))


In [ ]:
location_columns = [col for col in df.columns if 'location' in col]
print("Columns with 'location' in their name:")
num=0
for col in location_columns:
    print(col)
    num += 1
print("Total number of columns with location:", num)

In [ ]:
# EDA

#Identify all location related columns
location_cols = [c for c in df.columns if "location" in c.lower()]
print("Total location columns:", len(location_cols))


The dataset contains extremely high-dimensional location metadata, hence we need to reduce it.

After analysis the most required columns are the ones relating to location and rating, hence reducing to these columns.

In [ ]:
# Selecting Only Location columns and Rating columns
rating_cols = [
    "location.rating.cleanliness",
    "location.rating.facilities",
    "location.rating.services",
    "location.rating.average",
    "location.rating.value",
    "location.rating.count",
    "location.rating.bestrating"
]

# Filter columns that actually exist
rating_cols = [c for c in rating_cols if c in df.columns]

location_df = df[location_cols + rating_cols]
print("Filtered df:", location_df.shape)


This filtered dataframe contains the filtered columns further required which has details on location and ratings

In [ ]:
# Removing columns with very high number of missing values
missing_percent = location_df.isnull().mean().sort_values(ascending=False)
location_df = location_df.drop(columns=missing_percent[missing_percent > 0.80].index)

print("After removing >80% missing:", location_df.shape)


1. The reason for removing this above 80% missing value columns was because the original dataset has hundreds of nested columns, in which many are empty.

2. This reduction shows that most raw location columns are unusable, strengthening the case for creating an **aggregated representation**.

3. The **reason 1565 columns shrink to 89** is because

The dataset contains:

- 22 doctorlocations * ~70 repeated fields
- 9 doctorlocationhours * ~60 repeated fields
- city substructures * repeated image sizes (32*32, 70*70, 77*77, 165*165…)
- rating substructures
- hundreds of rarely-filled, API-generated metadata columns
- thousands of columns where 99% of values are empty

4. These were some of the **columns that are significant**:
- location.rating.count
- location.rating.bestrating
- location.rating.cleanliness
- location.rating.value
- location.rating.count
- location.rating.average
- location.rating.bestrating
- location.rating.services
- location.rating.facilities

5. We have **removed**:
- Broken / unused nested keys
- Empty image placeholders
- Missing city-level fields
- Duplicated doctorlocation structures
- Mostly-null geocode fields

In [ ]:
# Creating a copy of the final column
location_core = location_df.copy()
print("Final usable columns:", location_core.shape)

This is the final usable column

In [ ]:
location_core.columns

In [ ]:
# Dropping the columns with images
image_cols = ['location.images.autoxauto' , 'location.images.77x77' , 'location.images.100x100' , 'location.images.165x165' , 'location.images.70x70' , 'location.city.coverimages.253x83' , 'location.city.coverimages.autoxauto']
final_df = location_core.drop(columns=image_cols)
print("Final usable columns:", final_df.shape)

In [ ]:
# fixing the column types
final_df.dtypes

# Method 3: Location and Rating

In [1]:
df = gdrive_df

#dropping columns with rating and has only location
location_columns = [col for col in df.columns if 'location' in col]
location_only_cols = [col for col in location_columns if 'rating' not in col]
df_withlocation_norating = df[location_only_cols]
print("Filtered df:", df_withlocation_norating.shape)

# removing image columns
image_cols = [col for col in df_withlocation_norating.columns if 'images' in col]
final_df = df_withlocation_norating.drop(columns=image_cols)
print("Final usable columns:", final_df.shape)

NameError: name 'gdrive_df' is not defined

In [29]:
for col in final_df.columns:
  if final_df[col].dtype == 'float64':
    print(col)

location.id
location.longitude
location.latitude
location.city.id
location.city.province
location.city_id
doctor_locations[0].id
doctor_locations[1].id
doctor_locations[2].id
doctor_locations[3].id
doctor_locations[4].id
doctor_locations[5].id
doctor_locations[6].id
doctor_locations[7].id
doctor_locations[8].id
doctor_locations[9].id
doctor_locations[10].id
doctor_locations[11].id
doctor_locations[12].id
doctor_locations[13].id
doctor_locations[14].id
doctor_locations[15].id
doctor_locations[16].id
doctor_locations[17].id
doctor_locations[18].id
doctor_locations[19].id
doctor_locations[20].id
doctor_locations[21].id
doctor_locations[0].location.id
doctor_locations[1].location.id
doctor_locations[2].location.id
doctor_locations[3].location.id
doctor_locations[4].location.id
doctor_locations[5].location.id
doctor_locations[6].location.id
doctor_locations[7].location.id
doctor_locations[8].location.id
doctor_locations[9].location.id
doctor_locations[10].location.id
doctor_locations[11].lo

## Model Planning

## Model building / Analysis

## Discussion and Results

## Conclusion

## References

1. Barber, David. Bayesian Reasoning and Machine Learning. Cambridge University Press, 2012.
2. Aste, Tomaso, Paola Cerchiello, and Roberta Scaramozzino. "Information-Theoretic Causality Detection between Financial and Sentiment Data."Entropy, vol. 24, no. 6, 2022, pp. 1–18. DOI:10.3390/e24060774.
3. Metz, Cade. "Microsoft Puts OpenAI’s Sam Altman in Charge of New Advanced AI Research Team."
The New York Times, 20 Nov. 2023, www.nytimes.com/2023/11/20/technology/openai-microsoft-altman.html

## Appendix